## Linear Elasticity

In [1]:
import mfem.ser as mfem
import numpy as np
import time
from glvis import glvis

In [2]:
# Polynomial degree
order = 1

# Rectangular mesh
mesh = mfem.Mesh("beam-tri.mesh")

# Refine uniformly twice
mesh.UniformRefinement()
mesh.UniformRefinement()

# dim = 2 for this test case
dim = mesh.Dimension()

# H^1 finite elements, with order = 1
fec = mfem.H1_FECollection(order, dim)

# Create the finite element space. Note that dim for the third argument creates
# the space V_h = [W_h]^d, where W_h is the scalar space in H^1.
fes = mfem.FiniteElementSpace(mesh, fec, dim)

In [ ]:
mesh.SetNodalFESpace(fes)
orig_nodes = mfem.Vector(mesh.GetNodes())
glvis(mesh)

In [ ]:
print("Number of unknowns:", fes.GetVSize())

In [ ]:
# ess_bdr is an array of 1s and 0s. We put a 1 in the place where the boundary
# attribute is 'essential' (i.e. displacement, Dirichlet), and 0 in the place
# where the boundary is natural (i.e. traction, Neumann)
ess_bdr = mfem.intArray([1, 0, 0])

# Get the list of essential DOFs
ess_tdof_list = mfem.intArray()
fes.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

print("Number of essential BC DOFs:", ess_tdof_list.Size())

In [15]:
# Define the Lamé parameters (piecewise constants according to the two mesh
# attributes)
lambda_vec = mfem.Vector([2.0, 1.0])
lambda_coeff = mfem.PWConstCoefficient(lambda_vec)

mu_vec = mfem.Vector([2.0, 1.0])
mu_coeff = mfem.PWConstCoefficient(mu_vec)

In [16]:
# Create the elasticity bilinear form with given lambda and mu
a = mfem.BilinearForm(fes)
a.AddDomainIntegrator(mfem.ElasticityIntegrator(lambda_coeff, mu_coeff))
a.Assemble()

In [17]:
# Add the pull-down force on the right part of the boundary to the right-hand
# side vector
f = mfem.VectorArrayCoefficient(dim)

# There is no force anywhere in the x-direction
f.Set(0, mfem.ConstantCoefficient(0.0))

# In the y-direction, there is a force of 1e-2 downwards on boundary attribute 2
pull_force = mfem.Vector([0, -1e-2, 0])
f.Set(1, mfem.PWConstCoefficient(pull_force))

b = mfem.LinearForm(fes)
b.AddBoundaryIntegrator(mfem.VectorBoundaryLFIntegrator(f))
b.Assemble()

In [18]:
x = mfem.GridFunction(fes)

In [19]:
# Form the linear system: eliminate boundary conditions, etc.
A = mfem.SparseMatrix()
B = mfem.Vector()
X = mfem.Vector()
x.Assign(0.0)
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)

In [ ]:
# Gauss-Seidel preconditioner
M = mfem.GSSmoother(A)
# Preconditioned conjugate-gradient solver
mfem.PCG(A, M, B, X, 1, 500, 1e-8, 0.0)

In [21]:
# Recover the solution as a finite element grid function.
a.RecoverFEMSolution(X, b, x)

In [22]:
nodes = mesh.GetNodes()
nodes.Assign(orig_nodes)
nodes += x
y = mfem.GridFunction(fes)
y.Assign(x)
y *= -1.0

In [ ]:
glvis((mesh, y), keys="jR")